In [2]:
from pro.data.vocdata import VocDataset
label_dir = 'pro/data/dataset/VOCdevkit/VOC2007/Annotations'
image_dir = 'pro/data/dataset/VOCdevkit/VOC2007/JPEGImages'
voc_dataset = VocDataset(label_dir, image_dir)
print('datanumber: {}'.format(voc_dataset.__len__()))
# next(iter(voc_dataset))


datanumber: 5011


In [3]:
from pro.data.vocdata import VocDataloader
voc_dataloader= VocDataloader(voc_dataset)
train_dataloader, val_dataloader = voc_dataloader(0.7,batch_size_train=3,sample=100)

print('batch number of train_data: {},val_data: {}'.format(train_dataloader.__len__(),val_dataloader.__len__()))

batch number of train_data: 12,val_data: 16


In [4]:
from pro.model.model import FasterRcnn
from pro.util.utils import get_class_dict

class_name_dict = get_class_dict(label_dir)
print(class_name_dict)
fasterrcnn_cls = FasterRcnn(class_name_dict, device='cpu')

{0: 'diningtable', 1: 'chair', 2: 'horse', 3: 'person', 4: 'tvmonitor', 5: 'bird', 6: 'cow', 7: 'dog', 8: 'bottle', 9: 'pottedplant', 10: 'aeroplane', 11: 'car', 12: 'cat', 13: 'sheep', 14: 'bicycle', 15: 'sofa', 16: 'boat', 17: 'train', 18: 'motorbike', 19: 'bus'}


In [5]:
# fasterrcnn_cls.train(train_dataloader, num_epochs=1)

In [6]:
fasterrcnn_cls.eval(val_dataloader, 
            load_from='output/221216-175225/model/train_ALL_VOC2007.cpu.pt', 
            plot_result=True)

FileNotFoundError: [Errno 2] No such file or directory: 'output/221216-175225/train_ALL_VOC2007.cpu.pt'

In [7]:
from pathlib import Path 
a = Path('output_model/221216-175225/train_ALL_VOC2007.cpu.pt')
a.parent

PosixPath('output_model/221216-175225')